This is my first Release Code.

I learn from this [code](https://www.kaggle.com/josepc/rsna-effnet) so much,Thank you.

Please Run with My this code[[Train_with_TFRecord_for_RSNA-Radiogenomic](https://www.kaggle.com/hazigin/train-with-tfrecord-for-rsna-radiogenomic)]

In [ ]:
import numpy as np
from numpy.lib.function_base import append
import pandas as pd
import os
import sys
import tensorflow as tf
from pathlib import Path
import cv2
import pydicom
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
# limit the GPU memory growth
gpu = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpu))
if len(gpu) > 0:
    tf.config.experimental.set_memory_growth(gpu[0], True)

In [ ]:
# modelの設定
height = 512
width = 512
channel = 4
seed = 26
input_depth = 4

In [ ]:
inputdatapath = "../input/rsna-miccai-brain-tumor-radiogenomic-classification"
traindatapaht="../input/rsna-miccai-brain-tumor-radiogenomic-classification/test"
testdatapaht="../input/rsna-miccai-brain-tumor-radiogenomic-classification/train"
outpath = "./"

In [ ]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def serialize_example(feature0, feature2):
  feature = {
      'image': _bytes_feature(feature0.tobytes()),
      'MGMT_value': _float_feature(feature2)
  }
  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [ ]:
dim = (height,width)
def data_generation(ID):
    'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)

    # Store sample
    idx = str(ID).zfill(5)
    imgs = load_imgs(idx, ignore_zeros=False, train=True)
    new_imgs = []

    for ii in range(4):
        img_ = imgs[views[ii]].mean(axis=0)
        img_ = cv2.resize(img_, dsize=dim, interpolation=cv2.INTER_LINEAR)               
        new_imgs.append(img_)
    new_imgs = np.array(new_imgs).transpose(1,2,0)
    return new_imgs

In [ ]:
views = ['FLAIR', 'T1w', 'T1wCE', 'T2w']
def load_imgs(idx, ignore_zeros=True, train=True):
    imgs = {}
    for view in views:
        save_ds = []
        if train:
            dir_path = os.walk(os.path.join(
            traindatapaht, idx, view
        ))
        else:
            dir_path = os.walk(os.path.join(
            testdatapaht, idx, view
        ))
        for path, subdirs, files in dir_path:
            for name in files:
                image_path = os.path.join(path, name) 
                pyds = pydicom.filereader.dcmread(image_path)
                save_ds.append(np.array(pyds.pixel_array))
        if len(save_ds) == 0:
            save_ds = np.zeros((1,height,width))
        imgs[view] = np.array(save_ds)
    return imgs

In [ ]:
df_train = pd.read_csv(os.path.join(inputdatapath,"train_labels.csv"))
X_train, X_val, y_train, y_val = train_test_split(df_train.BraTS21ID, df_train.MGMT_value,
                                                 test_size=0.2, random_state=42,stratify=df_train.MGMT_value)

In [ ]:
with tf.io.TFRecordWriter(str(outpath + os.sep + 'brain_train.tfrec'),options=tf.io.TFRecordOptions(compression_type="GZIP")) as writer:
    for x,y in tqdm(zip(X_train,y_train)):
        img = data_generation(x)
        example = serialize_example(
            img, y)
        writer.write(example)


with tf.io.TFRecordWriter(str(outpath + os.sep + 'brain_val.tfrec'),options=tf.io.TFRecordOptions(compression_type="GZIP")) as writer:
    for x,y in tqdm(zip(X_val,y_val)):
        img = data_generation(x)
        example = serialize_example(
            img, y)
        writer.write(example)
